In [31]:
import os

import bk.load
import bk.compute
import bk.plot
import bk.signal

import numpy as np
import pandas as pd

import scipy.signal
import scipy.stats
import scipy.io

import matplotlib.pyplot as plt
import matplotlib as mpl
import neuroseries as nts

from tqdm import tqdm

In [35]:
def load_ripples():
    ripples_ = scipy.io.loadmat(f'{bk.load.session}-RippleFiring.mat')['ripples']['allsws'][0][0]
#     ripples_ = pd.DataFrame(data = ripples,columns=['start','peak','stop'])
    
    columns = ['start','peak','stop']
    
    ripples = {}
    for i,c in zip(range(ripples_.shape[1]),columns):
        ripples.update({c:nts.Ts(ripples_[:,i],time_units='s')})
    return ripples

def ripple_psth(nbins):
    ripples = load_ripples()
    neurons,metadata = bk.load.loadSpikeData(bk.load.path)

    win = [-0.5,0.5]
    n_neurons = len(neurons)
    
    peth = np.empty((len(ripples['peak'].as_units('s').index),100,n_neurons))
    
    for i,ripple_time in tqdm(enumerate(ripples['peak'].as_units('s').index)):
        t,binned = bk.compute.binSpikes(neurons,nbins=100,start = ripple_time+win[0],stop = ripple_time+win[1])
        peth[i,:] = binned.T


In [36]:
bk.load.current_session()
ripple_psth(100)

0it [00:00, ?it/s]

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


423it [00:38, 10.97it/s]


KeyboardInterrupt: 

In [5]:
def poissonTest(baseRate,counts,time):
    eps = np.spacing(1)
    
    lam = baseRate*time
    
    pInc = 1 - ss.poisson.cdf(counts,lam)
    pDec = ss.poisson.cdf(counts,lam)
    surprise = np.log((pDec+eps)/(pInc + eps))
    return pInc,pDec

In [28]:
states = bk.load.states()
ripples = load_ripples()
states.update({'ripples':nts.IntervalSet(ripples['start'].times(),ripples['stop'].times())})
# d.union(c).drop_short_intervals(1).as_units('s')
states.update({'outside_ripples':states['sws'].set_diff(nts.IntervalSet(states['ripples'].start-200_000,states['ripples'].end+200_000,force_no_fix=True))})


In [29]:
%matplotlib qt

In [30]:
s = plt.subplot(2,1,1)
bk.plot.intervals(states['sws'])
plt.subplot(2,1,2,sharex = s)
bk.plot.intervals(states['ripples'],'red')
bk.plot.intervals(states['outside_ripples'],'green')

,start,end
0,0.0,10.0
1,20.0,45.0
2,55.0,70.0


In [ ]:
# rip = states['sws'].intersect(states['ripples'])

In [ ]:
long_rip = states['ripples']

In [16]:
no_rip = states['sws'].set_diff(states['ripples'])

In [19]:
no_rip.as_units('s')

,start,end
0,1447.8000,1448.879200
1,1448.9472,1449.176000
2,1449.2152,1449.322399
3,1449.3584,1450.164000
4,1450.1968,1451.267200
...,...,...
5837,21830.2720,21836.876800
5838,21836.9144,21839.263200
5839,21839.3016,21839.656000
5840,21839.7224,21841.044000


In [161]:
n = neurons[1].restrict(states['sws'])

In [164]:
neurons/states['sws'].tot_length(time_units='s')

0.025843503230437905

252

In [129]:
poissonTest(15,5,5)

(1.0, 5.669857731738041e-26)

In [4]:
# t,spks_bin = bk.compute.binSpikes(neurons)
# binned = nts.TsdFrame(t,spks_bin.T,time_units='s')

In [97]:
import scipy.stats as ss

TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [61]:
for i in range(len(metadata[metadata['Region'] == 'Pir'])):
    plt.figure()
    plt.bar(t,np.mean(e,0)[:,metadata.Region == 'Pir'][:,i],width=t[1]-t[0])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  
